In [1]:
# Install required packages
%pip install -r requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [2]:
#dotenv module
from dotenv import load_dotenv
_ = load_dotenv()

In [3]:
#Adding typing module for typed python
from typing import TypedDict, Annotated
import operator

In [ ]:
#langgraph module and tavily search tool
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AnyMessage, SystemMessage, ToolMessage
from langchain_tavily import TavilySearch

In [ ]:
# Google Gemini used for this example : https://python.langchain.com/docs/integrations/providers/google/
from langchain_google_genai import ChatGoogleGenerativeAI

In [6]:
tool = TavilySearch(max_results=2)
print(type(tool))
print(tool.name)

<class 'langchain_tavily.tavily_search.TavilySearch'>
tavily_search


In [7]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [ ]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_gemini)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1] # An index of [-1] means "the last element (length - 1). Any time you have a negative index in Python, it means "that many from the length.
        return len(result.tool_calls) > 0

    def call_gemini(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}
    
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:    # check for bad tool name from LLM
                print("\n ....bad tool name, retry....")
                result = "bad tool name, retry" # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}


In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information.
You are allowed to make multiple calls (either together or in sequence).envOnly look up information when you are sure of what you want.
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
abot = Agent(model, [tool], system=prompt)